<a href="https://colab.research.google.com/github/hargagan/EDA-NYC-Taxi-Data-Analysis/blob/main/Brewery-dataset/EDA_Assg_Beer_Production_hargagan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Beer Production Analysis**

## Objective

In this case study you’ll be learning Exploratory Data Analytics (EDA) with the help of a dataset on beer production statistics in the US. This will enable you to understand why EDA is an important step in the process of Machine Learning.

## Tasks

You need to perform the following steps for successfully completing this assignment.
1. Data Loading
2. Data Cleaning
3. Exploratory Analysis: Bivariate and Multivariate
4. Creating Visualisations to Support the Analysis
5. Deriving Insights and Stating Conclusions

## Data Understanding

The data comes from the Alcohol and Tobacco Tax and Trade Bureau of USA(TTB). You can find the source to the repository containing the data here: [Beer Production GitHub](https://github.com/rfordatascience/tidytuesday/tree/master/data/2020/2020-03-31)

<br>
The dataset contains three tables, in which you can find these details:

* `beer_states.csv` State-level beer production by year (2008-2019)
* `brewer_size.csv` Number of brewers by production size by year (2008-2019)
* `brewing_materials.csv` Monthly beer stats aggregated across the US (2008-2017)


Some considerations:
* A barrel of beer for this data is 31 gallons
* Removals = "Total barrels removed subject to tax by the breweries comprising the named strata of data", essentially how much was produced and removed for consumption.

### Data Description

`brewing_materials.csv`

|variable         |class     |description |
|:----------------|:---------|:-----------|
|data_type        |character | Pounds of Material|
|material_type    |character | Grain product, Totals, Non-Grain Product (basically hops vs grains)|
|year             |double    | Year |
|month            |integer   | Month |
|type             |character | Actual line-item from material type |
|month_current    |double    | Current number of barrels for this year/month |
|month_prior_year |double    | Prior year number of barrels for same month |
|ytd_current      |double    | Cumulative year to date of current year |
|ytd_prior_year   |double    | Cumulative year to date for prior year |

<br>

`beer_states.csv`

|variable |class     |description |
|:--------|:---------|:-----------|
|state    |character | State abbreviated |
|year     |integer   | Year |
|barrels  |double    | Barrels produced within each type |
|type     |character | Type of production/use (On premise, Bottles/Cans, Kegs/Barrels) |

<br>

`brewer_size.csv`

|variable         |class     |description |
|:----------------|:---------|:-----------|
|year             |integer   | Year  |
|brewer_size      |character | Range of production for brewer size, number of barrels produced |
|n_of_brewers     |double    | Number of brewers at that brewer size |
|total_barrels    |double    | Total barrels of beer produced at that brewer size |
|taxable_removals |double    | Taxable barrels for removals - removals for consumption under taxation |
|total_shipped    |double    | Total barrels shipped - produced beer that is not taxed |

**NOTE**: The marks given along with headings and sub-headings are cumulative marks for those particular headings/sub-headings.<br>
The actual marks for each task are specified within the tasks themselves.

For example, marks given with heading *2* or sub-heading *2.1* are the cumulative marks, for your reference only. <br>

The marks you will receive for completing tasks are given with the tasks. Suppose the marks for two tasks are as follows:
* 2.1.1 [3 marks]
* 3.2.2 [2 marks]

You will earn 3 marks for completing task 2.1.1 and 2 marks for completing task 3.2.2.

### Import Libraries and Load Dataset

In [1]:
# Import warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import the libraries you will be using for analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Load the three csv files
states = "/content/drive/MyDrive/Assignments/EDA/Brewery-Datasets/beer_states.csv"
brewers = "/content/drive/MyDrive/Assignments/EDA/Brewery-Datasets/brewer_size.csv"
materials = "/content/drive/MyDrive/Assignments/EDA/Brewery-Datasets/brewing_materials.csv"

In [5]:
# Take a look at the dataframes and understand the structure
df_states = pd.read_csv(states)
df_states.head()



,state,year,barrels,type
0,AK,2008,2067.69,On Premises
1,AK,2009,2263.65,On Premises
2,AK,2010,1929.15,On Premises
3,AK,2011,2251.02,On Premises
4,AK,2012,2312.43,On Premises


In [6]:
df_states.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1872 entries, 0 to 1871
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   state    1872 non-null   object 
 1   year     1872 non-null   int64  
 2   barrels  1853 non-null   float64
 3   type     1872 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 58.6+ KB


In [7]:
df_states.shape

(1872, 4)

In [8]:
df_brewers = pd.read_csv(brewers)
df_brewers.head()

,year,brewer_size,n_of_brewers,total_barrels,taxable_removals,total_shipped
0,2009,"6,000,001 Barrels and Over",18,1.712329e+08,1.596440e+08,3639970.0
1,2009,"1,000,001 to 6,000,000 Barrels",4,9.970404e+06,9.592723e+06,14548.0
2,2009,"500,001 to 1,000,000 Barrels",7,4.831386e+06,4.535659e+06,21563.0
3,2009,"100,001 to 500,000 Barrels",27,5.422156e+06,4.469289e+06,128000.0
4,2009,"60,001 to 100,000 Barrels",19,1.501977e+06,1.224618e+06,95732.0


In [9]:
df_brewers.shape

(137, 6)

In [10]:
df_brewers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   year              137 non-null    int64  
 1   brewer_size       137 non-null    object 
 2   n_of_brewers      137 non-null    int64  
 3   total_barrels     136 non-null    float64
 4   taxable_removals  137 non-null    float64
 5   total_shipped     135 non-null    float64
dtypes: float64(3), int64(2), object(1)
memory usage: 6.6+ KB


In [11]:
df_materials = pd.read_csv(materials)
df_materials.head()

,data_type,material_type,year,month,type,month_current,month_prior_year,ytd_current,ytd_prior_year
0,Pounds of Materials Used,Grain Products,2008,1,Malt and malt products,374165152,365300134,374165152.0,365300134.0
1,Pounds of Materials Used,Grain Products,2008,1,Corn and corn products,57563519,41647092,57563519.0,41647092.0
2,Pounds of Materials Used,Grain Products,2008,1,Rice and rice products,72402143,81050102,72402143.0,81050102.0
3,Pounds of Materials Used,Grain Products,2008,1,Barley and barley products,3800844,2362162,3800844.0,2362162.0
4,Pounds of Materials Used,Grain Products,2008,1,Wheat and wheat products,1177186,1195381,1177186.0,1195381.0


In [12]:
df_materials.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1440 entries, 0 to 1439
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   data_type         1440 non-null   object 
 1   material_type     1440 non-null   object 
 2   year              1440 non-null   int64  
 3   month             1440 non-null   int64  
 4   type              1440 non-null   object 
 5   month_current     1440 non-null   int64  
 6   month_prior_year  1440 non-null   int64  
 7   ytd_current       1008 non-null   float64
 8   ytd_prior_year    1008 non-null   float64
dtypes: float64(2), int64(4), object(3)
memory usage: 101.4+ KB


In [76]:
df_materials.loc[df_materials['ytd_current'].isna()].head()

,data_type,material_type,year,month,type,month_current,month_prior_year,ytd_current,ytd_prior_year
1008,Pounds of Materials Used,Grain Products,2015,1,Malt and malt products,314189672,315728713,NaN,NaN
1009,Pounds of Materials Used,Grain Products,2015,1,Corn and corn products,59759340,60186876,NaN,NaN
1010,Pounds of Materials Used,Grain Products,2015,1,Rice and rice products,52438465,52099471,NaN,NaN
1011,Pounds of Materials Used,Grain Products,2015,1,Barley and barley products,8728262,8185386,NaN,NaN
1012,Pounds of Materials Used,Grain Products,2015,1,Wheat and wheat products,1316582,1372263,NaN,NaN


In [13]:
df_materials.shape

(1440, 9)

In [89]:
def check_null_columns(df):
    null_columns = df.columns[df.isnull().any()]
    print("Number of null columns in the dataframe: " + str(len(null_columns)))
    return df[null_columns].isnull().mean()*100

def check_str_total_in_columns(df):
  cols = df.columns
  out = " "
  for col in cols:
    # Check if the data type of the col is string else ignore
    if df[col].dtype == 'object' and df[col].str.contains('total', case=False).sum() > 0 :
      out += f" [{col} - {df[col].str.contains('total', case=False).sum()}] - \nUnique_vals: {df[col].unique()}\n"
  return out

def col_unique_vals_greater_than_1(df):
  cols = df.columns
  out = " "
  for col in cols:
    # Check if the data type of the col is string else ignore
    if df[col].dtype == 'object' and df[col].nunique() >= 1 :
      out += f" [{col} - {df[col].nunique()}] - \nUnique_vals: {df[col].unique()}\n"
  return out

In [53]:
df_materials['type'].str.contains('Total', case=False).sum()

np.int64(360)

In [15]:
(df_states['state']=='total').sum()

np.int64(36)

In [34]:
df_states.head()

,state,year,barrels,type
0,AK,2008,2067.69,On Premises
1,AK,2009,2263.65,On Premises
2,AK,2010,1929.15,On Premises
3,AK,2011,2251.02,On Premises
4,AK,2012,2312.43,On Premises


In [35]:
print(df_states['type'].unique())

array(['On Premises', 'Bottles and Cans', 'Kegs and Barrels'],
      dtype=object)

In [80]:
df_materials.head()
print(df_materials['type'].unique())
print(df_materials['material_type'].unique())
print(df_materials['data_type'].unique())

['Malt and malt products' 'Corn and corn products'
 'Rice and rice products' 'Barley and barley products'
 'Wheat and wheat products' 'Total Grain products' 'Sugar and syrups'
 'Hops (dry)' 'Hops (used as extracts)' 'Other' 'Total Non-Grain products'
 'Total Used']
['Grain Products' 'Total Grain products' 'Non-Grain Products'
 'Total Non-Grain products' 'Total Used']
['Pounds of Materials Used']


In [106]:
print(df_brewers['brewer_size'].unique())

# df_brewers having brewer_size equal 'Zero Barrels'. It is only single row has 'total_shipped' also mentioned as 'NaN'
print('\nZero Barrel')
display(df_brewers[df_brewers['brewer_size'] == 'Zero Barrels'])

print("\nUnder 1 barrel")
display(df_brewers[df_brewers['brewer_size'] == 'Under 1 Barrel'])

print ("Total Barrel")
display(df_brewers.loc[(df_brewers['brewer_size'] == 'Total') & (df_brewers['year'] == 2009)])
df_brewers.loc[df_brewers['brewer_size'] != 'Total'].groupby('year')['total_barrels'].sum()


['6,000,001 Barrels and Over' '1,000,001 to 6,000,000 Barrels'
 '500,001 to 1,000,000 Barrels' '100,001 to 500,000 Barrels'
 '60,001 to 100,000 Barrels' '30,001 to 60,000 Barrels'
 '15,001 to 30,000 Barrels' '7,501 to 15,000 Barrels'
 '1,001 to 7,500 Barrels' '1 to 1,000 Barrels' 'Under 1 Barrel' 'Total'
 '1,000,000 to 6,000,000 Barrels' '2,000,000 to 6,000,000 Barrels'
 '1,000,001 to 1,999,999 Barrels' 'Zero Barrels']

Zero Barrel


,year,brewer_size,n_of_brewers,total_barrels,taxable_removals,total_shipped
82,2015,Zero Barrels,476,0.0,87811.22,NaN



Under 1 barrel


,year,brewer_size,n_of_brewers,total_barrels,taxable_removals,total_shipped
10,2009,Under 1 Barrel,195,3.70,14711.36,119.0
95,2016,Under 1 Barrel,488,4.43,102983.40,601.0
108,2017,Under 1 Barrel,553,6.73,107494.10,503.0
121,2018,Under 1 Barrel,164,7.47,142501.39,274.0
134,2019,Under 1 Barrel,28,18.30,19432.26,NaN
135,2019,Under 1 Barrel,174,NaN,114960.27,185.0


Total Barrel


,year,brewer_size,n_of_brewers,total_barrels,taxable_removals,total_shipped
11,2009,Total,1778,196969275.0,1.829955e+08,3934233.0


,total_barrels
year,
2009,1.969693e+08
2010,1.952114e+08
2011,1.927468e+08
2012,1.961472e+08
2013,1.919974e+08
2014,1.920151e+08
2015,1.911130e+08
2016,1.898399e+08
2017,1.855818e+08


In [109]:

# Get total barrels from df_states for each year (where state is 'total')
# The df_states 'total' rows provide totals for each 'type' (On Premises, Bottles and Cans, Kegs and Barrels).
# To get the grand total barrels for a year, we need to sum these 'type' totals.
#total_barrels_from_states = df_states[df_states['state'] == 'total'].groupby('year')['barrels'].sum()

# Get total barrels from df_brewers for each year where brewer_size is 'Total'
# This aggregates the total barrels produced by all brewers for each year.
total_barrels_from_brewers_total_size = df_brewers[df_brewers['brewer_size'] == 'Total'].groupby('year')['total_barrels'].sum()

# Compare with the user's specific request for df_brewers
# 1. Get the 'Total' rows for each year
df_brewers_total_rows = df_brewers[df_brewers['brewer_size'] == 'Total'].set_index('year')

# 2. Get the sum of all other brewer_size categories for each year
df_brewers_sum_other_rows = df_brewers[df_brewers['brewer_size'] != 'Total'].groupby('year')[['total_barrels', 'taxable_removals', 'total_shipped']].sum()

print("\nComparison of 'Total' brewer_size values with sum of other brewer_sizes:")

comparison_fields = ['total_barrels', 'taxable_removals', 'total_shipped']
for field in comparison_fields:
    print(f"\nComparing {field}:")
    combined_comparison = pd.DataFrame({
        f'Total_brewer_size_{field}': df_brewers_total_rows[field],
        f'Sum_of_other_brewer_sizes_{field}': df_brewers_sum_other_rows[field]
    })

    # Check for equality, rounding to avoid floating point issues
    equality_check = (combined_comparison[f'Total_brewer_size_{field}'].round(2) == combined_comparison[f'Sum_of_other_brewer_sizes_{field}'].round(2))
    if equality_check.all():
        print(f"All {field} values are equal.")
    else:
        print(f"Not all {field} values are equal.")
        combined_comparison['difference'] = combined_comparison[f'Total_brewer_size_{field}'] - combined_comparison[f'Sum_of_other_brewer_sizes_{field}']
    display(combined_comparison)
    print(f"Are {field} values equal (rounded to 2 decimal places)?")
    display(equality_check.value_counts())



Comparison of 'Total' brewer_size values with sum of other brewer_sizes:

Comparing total_barrels:
All total_barrels values are equal.


,Total_brewer_size_total_barrels,Sum_of_other_brewer_sizes_total_barrels
year,,
2009,1.969693e+08,1.969693e+08
2010,1.952114e+08,1.952114e+08
2011,1.927468e+08,1.927468e+08
2012,1.961472e+08,1.961472e+08
2013,1.919974e+08,1.919974e+08
2014,1.920151e+08,1.920151e+08
2015,1.911130e+08,1.911130e+08
2016,1.898399e+08,1.898399e+08
2017,1.855818e+08,1.855818e+08


Are total_barrels values equal (rounded to 2 decimal places)?


,count
True,11



Comparing taxable_removals:
All taxable_removals values are equal.


...

,count
True,11



Comparing total_shipped:
Not all total_shipped values are equal.


,Total_brewer_size_total_shipped,Sum_of_other_brewer_sizes_total_shipped,difference
year,,,
2009,3934233.0,3934484.0,-251.0
2010,4050445.0,4050442.0,3.0
2011,4426027.0,4426023.0,4.0
2012,5031215.0,5031212.0,3.0
2013,5311134.0,5311129.0,5.0
2014,5508662.0,5508656.0,6.0
2015,6211297.0,6211292.0,5.0
2016,6490978.0,6490972.0,6.0
2017,6814178.0,6814173.0,5.0


Are total_shipped values equal (rounded to 2 decimal places)?


,count
False,11


## **1** Data Cleaning
<font color = red>[20 marks]</font> <br>

The data is mostly cleaned, but you may find a few anomalies.

#### **1.1** Fixing Columns
<font color = red>[5 marks]</font> <br>
Fix/drop any columns as you seem necessary.

##### **1.1.1** *states* table

In [91]:
# Check if there are any columns that can be dropped/fixed.
print(check_null_columns(df_states))
print(check_str_total_in_columns(df_states))
print(col_unique_vals_greater_than_1(df_states))

Number of null columns in the dataframe: 1
barrels    1.014957
dtype: float64
  [state - 36] - 
Unique_vals: ['AK' 'AL' 'AR' 'AZ' 'CA' 'CO' 'CT' 'DC' 'DE' 'FL' 'GA' 'HI' 'IA' 'ID'
 'IL' 'IN' 'KS' 'KY' 'LA' 'MA' 'MD' 'ME' 'MI' 'MN' 'MO' 'MS' 'MT' 'NC'
 'ND' 'NE' 'NH' 'NJ' 'NM' 'NV' 'NY' 'OH' 'OK' 'OR' 'PA' 'RI' 'SC' 'SD'
 'TN' 'TX' 'UT' 'VA' 'VT' 'WA' 'WI' 'WV' 'WY' 'total']

  [state - 52] - 
Unique_vals: ['AK' 'AL' 'AR' 'AZ' 'CA' 'CO' 'CT' 'DC' 'DE' 'FL' 'GA' 'HI' 'IA' 'ID'
 'IL' 'IN' 'KS' 'KY' 'LA' 'MA' 'MD' 'ME' 'MI' 'MN' 'MO' 'MS' 'MT' 'NC'
 'ND' 'NE' 'NH' 'NJ' 'NM' 'NV' 'NY' 'OH' 'OK' 'OR' 'PA' 'RI' 'SC' 'SD'
 'TN' 'TX' 'UT' 'VA' 'VT' 'WA' 'WI' 'WV' 'WY' 'total']
 [type - 3] - 
Unique_vals: ['On Premises' 'Bottles and Cans' 'Kegs and Barrels']



##### **1.1.2** *materials* table

In [116]:
# Check if there are any columns that can be dropped/fixed.

print("\nChecking null columns in df_materials \n")
print(check_null_columns(df_materials))
print(check_str_total_in_columns(df_materials))
print(col_unique_vals_greater_than_1(df_materials))




Checking null columns in df_materials 

Number of null columns in the dataframe: 2
ytd_current       30.0
ytd_prior_year    30.0
dtype: float64
  [material_type - 360] - 
Unique_vals: ['Grain Products' 'Total Grain products' 'Non-Grain Products'
 'Total Non-Grain products' 'Total Used']
 [type - 360] - 
Unique_vals: ['Malt and malt products' 'Corn and corn products'
 'Rice and rice products' 'Barley and barley products'
 'Wheat and wheat products' 'Total Grain products' 'Sugar and syrups'
 'Hops (dry)' 'Hops (used as extracts)' 'Other' 'Total Non-Grain products'
 'Total Used']

  [material_type - 5] - 
Unique_vals: ['Grain Products' 'Total Grain products' 'Non-Grain Products'
 'Total Non-Grain products' 'Total Used']
 [type - 12] - 
Unique_vals: ['Malt and malt products' 'Corn and corn products'
 'Rice and rice products' 'Barley and barley products'
 'Wheat and wheat products' 'Total Grain products' 'Sugar and syrups'
 'Hops (dry)' 'Hops (used as extracts)' 'Other' 'Total Non-Grain pr

In [94]:
df_materials = df_materials.drop(columns=['data_type'])

##### **1.1.3** *brewers* table

In [95]:
# Check if there are any columns that can be dropped/fixed.

print("\nChecking null columns in df_brewers")
print(check_null_columns(df_brewers))
print(check_str_total_in_columns(df_brewers))
print(col_unique_vals_greater_than_1(df_brewers))


Checking null columns in df_brewers
Number of null columns in the dataframe: 2
total_barrels    0.729927
total_shipped    1.459854
dtype: float64
  [brewer_size - 11] - 
Unique_vals: ['6,000,001 Barrels and Over' '1,000,001 to 6,000,000 Barrels'
 '500,001 to 1,000,000 Barrels' '100,001 to 500,000 Barrels'
 '60,001 to 100,000 Barrels' '30,001 to 60,000 Barrels'
 '15,001 to 30,000 Barrels' '7,501 to 15,000 Barrels'
 '1,001 to 7,500 Barrels' '1 to 1,000 Barrels' 'Under 1 Barrel' 'Total'
 '1,000,000 to 6,000,000 Barrels' '2,000,000 to 6,000,000 Barrels'
 '1,000,001 to 1,999,999 Barrels' 'Zero Barrels']

  [brewer_size - 16] - 
Unique_vals: ['6,000,001 Barrels and Over' '1,000,001 to 6,000,000 Barrels'
 '500,001 to 1,000,000 Barrels' '100,001 to 500,000 Barrels'
 '60,001 to 100,000 Barrels' '30,001 to 60,000 Barrels'
 '15,001 to 30,000 Barrels' '7,501 to 15,000 Barrels'
 '1,001 to 7,500 Barrels' '1 to 1,000 Barrels' 'Under 1 Barrel' 'Total'
 '1,000,000 to 6,000,000 Barrels' '2,000,000 to 6

#### **1.2** Fixing Rows
<font color = red>[5 marks]</font> <br>
Can you find rows having totals in any table? To avoid these from reflecting in the analyses, you can drop these.

In [ ]:
# Remove rows having totals below.


##### **1.2.1** *states* table

In [135]:
# Remove rows having totals.
print(f"Before cleaning {len(df_states['state'].unique())}")
df_states_clean = df_states[~(df_states['state'] == 'total')]
print(f"Before cleaning {len(df_states_clean['state'].unique())}")
df_states_clean['state'].unique()

Before cleaning 52
Before cleaning 51


array(['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
       'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
       'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
       'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY'], dtype=object)

##### **1.2.2** *materials* table

In [128]:
# Remove rows having totals

# [material_type - 5] -  'Total Grain products' 'Total Non-Grain products' 'Total Used'
# [type - 12] - 'Total Grain products' 'Total Non-Grain products' 'Total Used'
# Both type and material_type contains the string 'Total' in the same row. So taking care of one takes care of other
# as well.
print(f"Before cleaning {len(df_materials['type'].unique())}")
df_materials_clean = df_materials[~df_materials['material_type'].str.contains('Total', case=False)]
#df_material_clean['material_type'].unique()


print(f"After cleaning {len(df_materials_clean['type'].unique())} - (3 unique-values with 'Total' in them, are gone)")

Before cleaning 12
After cleaning 9 - (3 unique-values with 'Total' in them, are gone)


##### **1.2.3** *brewers* table

In [129]:
# Remove rows having totals

print(f"Before cleaning - {len(df_brewers['brewer_size'].unique())}")

df_brewers_clean = df_brewers[df_brewers['brewer_size'] != 'Total']

print(f"After cleaning - {len(df_brewers_clean['brewer_size'].unique())}")

Before cleaning - 16
After cleaning - 15


### **1.3** Handling Missing Values
<font color = red>[5 marks]</font> <br>

##### **1.3.1** *states*
<font color = red>[1 marks]</font> <br>

In [148]:
# Find the proportion of missing values in each column
df_states.info()
# barrels column are having null in around 19 columns.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1872 entries, 0 to 1871
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   state    1872 non-null   object 
 1   year     1872 non-null   int64  
 2   barrels  1853 non-null   float64
 3   type     1872 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 58.6+ KB


,0
state,0.000000
year,0.000000
barrels,1.034858
type,0.000000


In [239]:
df_states_clean.loc[df_states_clean['barrels'].isna()]
# df_states[df_states['state']=='total']
#df_states.loc[df_states['state']=='ND']['barrels'].isna().sum()
# Out of 19 NaN entries in barrels column, 10 are from ND - North Dakota

,state,year,barrels,type
300,MS,2008,NaN,On Premises
636,AL,2008,NaN,Bottles and Cans
637,AL,2009,NaN,Bottles and Cans
638,AL,2010,NaN,Bottles and Cans
960,ND,2008,NaN,Bottles and Cans
961,ND,2009,NaN,Bottles and Cans
962,ND,2010,NaN,Bottles and Cans
963,ND,2011,NaN,Bottles and Cans
964,ND,2012,NaN,Bottles and Cans
1116,SD,2008,NaN,Bottles and Cans


# All the data missing is from 2008 to 2012

In [240]:
missing_years = df_states_clean.loc[df_states_clean['barrels'].isna(), 'year'].unique()
missing_years

array([2008, 2009, 2010, 2011, 2012])

In [254]:
# Get the mean and mode barrel data for 2008 grouped by all different types
df_states_clean.loc[(df_states_clean['year'] == 2008)].groupby('type')['barrels'].agg(['mean', 'median']).reset_index()
#df_states_clean.groupby(['year', 'type'])['barrels'].agg(['mean', 'median']).reset_index()

,type,mean,median
0,Bottles and Cans,3.477709e+06,55340.140
1,Kegs and Barrels,3.527688e+05,28163.290
2,On Premises,9.342961e+03,6616.645


In [ ]:
# Analyse the null values and decide what to do with them



##### **1.3.2** *materials*
<font color = red>[2 marks]</font> <br>

In [158]:
# Find the percentage of missing values in each column
# df_materials_clean.info()
df_materials_clean.isna().mean()*100


,0
material_type,0.0
year,0.0
month,0.0
type,0.0
month_current,0.0
month_prior_year,0.0
ytd_current,30.0
ytd_prior_year,30.0


There is a significant amount of missing values in some columns. Check the rows with null values.

In [168]:
print(df_materials_clean.loc[df_materials_clean['ytd_current'].isna()]['year'].unique())
print(df_materials_clean.loc[df_materials_clean['ytd_current'].isna()]['month'].unique())
print(df_materials_clean.loc[df_materials_clean['ytd_prior_year'].isna()]['year'].unique())
print(df_materials_clean.loc[df_materials_clean['ytd_prior_year'].isna()]['month'].unique())

[2015 2016 2017]
[ 1  2  3  4  5  6  7  8  9 10 11 12]
[2015 2016 2017]
[ 1  2  3  4  5  6  7  8  9 10 11 12]


**Analysis**

#### The columns - ytd_current and ytd_prior_year missing from 2015 to 2017.

In [205]:
def get_materials_month_1(year) :
  return df_materials.loc[(df_materials['year'] == year) &
 (df_materials['material_type'].isin(['Grain Products', 'Total Grain products'])) &
 #(df_materials['type'] == 'Malt and malt products') &
  (df_materials['month'] == 1)][[ 'month_current', 'month_prior_year', 'ytd_current', 'ytd_prior_year']]

print(f"2012: {get_materials_month_1(2012)} \n")
print(f"2013: {get_materials_month_1(2013)} \n")
print(f"2014: {get_materials_month_1(2014)} \n")
print(f"2015: {get_materials_month_1(2015)} \n")


2012:      month_current  month_prior_year  ytd_current  ytd_prior_year
576      315214803         323332497  315214803.0     323332497.0
577       61859978          58875604   61859978.0      58875604.0
578       63568058         108570592   63568058.0     108570592.0
579        5391781           5265988    5391781.0       5265988.0
580        1319508           1232206    1319508.0       1232206.0
581      447354128         497276887  447354128.0     497276887.0 

2013:      month_current  month_prior_year  ytd_current  ytd_prior_year
720      326450615         322367885  326450615.0     322367885.0
721       50597106          61859978   50597106.0      61859978.0
722       62177066          63568058   62177066.0      63568058.0
723        7749970           5830442    7749970.0       5830442.0
724        1537526           1345768    1537526.0       1345768.0
725      448512283         454972131  448512283.0     454972131.0 

2014:      month_current  month_prior_year  ytd_current  ytd

In [208]:
# 2014:      month_current  month_prior_year  ytd_current  ytd_prior_year
# 864      315728713         327069448  315728713.0     327069448.0
# 865       60186876          50616106   60186876.0      50616106.0
# 866       52099471          62177066   52099471.0      62177066.0
# 867        8185386           8196315    8185386.0       8196315.0
# 868        1372263           1544976    1372263.0       1544976.0
# 869      437572709         449603911  437572709.0     449603911.0

# Checking where the value of 327069448 (month_prior_year) value in 2014 exist somewhere in 2013 data ??
df_materials.loc[(df_materials['year'] == 2013) &
 (df_materials['month_current'] == 327069448)]

,material_type,year,month,type,month_current,month_prior_year,ytd_current,ytd_prior_year


In [204]:
df_materials.loc[(df_materials['year'] == 2014) & (df_materials['month'] == 1)].groupby('material_type').sum()

,year,month,type,month_current,month_prior_year,ytd_current,ytd_prior_year
material_type,,,,,,,
Grain Products,10070,5,Malt and malt productsCorn and corn productsRi...,437572709,449603911,437572709.0,449603911.0
Non-Grain Products,8056,4,Sugar and syrupsHops (dry)Hops (used as extrac...,85198621,93064857,85198621.0,93064857.0
Total Grain products,2014,1,Total Grain products,437572709,449603911,437572709.0,449603911.0
Total Non-Grain products,2014,1,Total Non-Grain products,85198621,93064857,85198621.0,93064857.0
Total Used,2014,1,Total Used,522771330,542668768,522771330.0,542668768.0


In [196]:
df_materials.loc[(df_materials['year'] == 2012) &
 (df_materials['material_type'] == 'Grain Products') &
 (df_materials['type'] == 'Malt and malt products') &
  (df_materials['month'] == 1)]

,material_type,year,month,type,month_current,month_prior_year,ytd_current,ytd_prior_year
576,Grain Products,2012,1,Malt and malt products,315214803,323332497,315214803.0,323332497.0


In [194]:
df_materials.loc[(df_materials['year'] == 2013) &
 (df_materials['material_type'] == 'Grain Products') &
 (df_materials['type'] == 'Malt and malt products') &
  (df_materials['month'] == 1)]

,material_type,year,month,type,month_current,month_prior_year,ytd_current,ytd_prior_year
720,Grain Products,2013,1,Malt and malt products,326450615,322367885,326450615.0,322367885.0


In [182]:
df_materials.loc[(df_materials['year'] == 2012) & (df_materials['material_type'].str.contains('Grain')) & (df_materials['month'] == 1)]

,material_type,year,month,type,month_current,month_prior_year,ytd_current,ytd_prior_year
576,Grain Products,2012,1,Malt and malt products,315214803,323332497,315214803.0,323332497.0
577,Grain Products,2012,1,Corn and corn products,61859978,58875604,61859978.0,58875604.0
578,Grain Products,2012,1,Rice and rice products,63568058,108570592,63568058.0,108570592.0
579,Grain Products,2012,1,Barley and barley products,5391781,5265988,5391781.0,5265988.0
580,Grain Products,2012,1,Wheat and wheat products,1319508,1232206,1319508.0,1232206.0
581,Total Grain products,2012,1,Total Grain products,447354128,497276887,447354128.0,497276887.0
582,Non-Grain Products,2012,1,Sugar and syrups,62988309,73785535,62988309.0,73785535.0
583,Non-Grain Products,2012,1,Hops (dry),9562820,6073312,9562820.0,6073312.0
584,Non-Grain Products,2012,1,Hops (used as extracts),293488,296621,293488.0,296621.0
585,Non-Grain Products,2012,1,Other,8994300,8177092,8994300.0,8177092.0


In [181]:
df_materials.loc[(df_materials['year'] == 2013) & (df_materials['material_type'].str.contains('Grain')) & (df_materials['month'] == 1)]

,material_type,year,month,type,month_current,month_prior_year,ytd_current,ytd_prior_year
720,Grain Products,2013,1,Malt and malt products,326450615,322367885,326450615.0,322367885.0
721,Grain Products,2013,1,Corn and corn products,50597106,61859978,50597106.0,61859978.0
722,Grain Products,2013,1,Rice and rice products,62177066,63568058,62177066.0,63568058.0
723,Grain Products,2013,1,Barley and barley products,7749970,5830442,7749970.0,5830442.0
724,Grain Products,2013,1,Wheat and wheat products,1537526,1345768,1537526.0,1345768.0
725,Total Grain products,2013,1,Total Grain products,448512283,454972131,448512283.0,454972131.0
726,Non-Grain Products,2013,1,Sugar and syrups,72878590,63306838,72878590.0,63306838.0
727,Non-Grain Products,2013,1,Hops (dry),9164708,9657602,9164708.0,9657602.0
728,Non-Grain Products,2013,1,Hops (used as extracts),323285,293488,323285.0,293488.0
729,Non-Grain Products,2013,1,Other,10673797,9176359,10673797.0,9176359.0


In [223]:
df_materials_clean.loc[(df_materials['year'] == 2014) & (df_materials_clean['material_type'].str.contains('Grain')) & (df_materials_clean['month'].isin([1, 2]))]

,material_type,year,month,type,month_current,month_prior_year,ytd_current,ytd_prior_year
864,Grain Products,2014,1,Malt and malt products,315728713,327069448,315728713.0,327069448.0
865,Grain Products,2014,1,Corn and corn products,60186876,50616106,60186876.0,50616106.0
866,Grain Products,2014,1,Rice and rice products,52099471,62177066,52099471.0,62177066.0
867,Grain Products,2014,1,Barley and barley products,8185386,8196315,8185386.0,8196315.0
868,Grain Products,2014,1,Wheat and wheat products,1372263,1544976,1372263.0,1544976.0
870,Non-Grain Products,2014,1,Sugar and syrups,72039259,72879470,72039259.0,72879470.0
871,Non-Grain Products,2014,1,Hops (dry),1890937,9187873,1890937.0,9187873.0
872,Non-Grain Products,2014,1,Hops (used as extracts),298281,323622,298281.0,323622.0
873,Non-Grain Products,2014,1,Other,10970144,10673892,10970144.0,10673892.0
876,Grain Products,2014,2,Malt and malt products,293750079,312151626,609795673.0,639221074.0


In [214]:
# Display the rows with null values
df_materials_clean.loc[(df_materials['year'] == 2015) & (df_materials_clean['material_type'].str.contains('Grain')) & (df_materials_clean['month'] == 1)]
#df_materials_clean.groupby('year')['month_current'].sum()
#df_materials_clean.groupby('year')['type'].sum().head()


,material_type,year,month,type,month_current,month_prior_year,ytd_current,ytd_prior_year
1008,Grain Products,2015,1,Malt and malt products,314189672,315728713,NaN,NaN
1009,Grain Products,2015,1,Corn and corn products,59759340,60186876,NaN,NaN
1010,Grain Products,2015,1,Rice and rice products,52438465,52099471,NaN,NaN
1011,Grain Products,2015,1,Barley and barley products,8728262,8185386,NaN,NaN
1012,Grain Products,2015,1,Wheat and wheat products,1316582,1372263,NaN,NaN
1014,Non-Grain Products,2015,1,Sugar and syrups,71798065,72039259,NaN,NaN
1015,Non-Grain Products,2015,1,Hops (dry),1899000,1890937,NaN,NaN
1016,Non-Grain Products,2015,1,Hops (used as extracts),298514,298281,NaN,NaN
1017,Non-Grain Products,2015,1,Other,11573104,10970144,NaN,NaN


What did you find? Do you think it's beneficial to impute/drop these values? Why? Why not?

Try to find more about the missing values? Why are these missing? Can you fix those easily? How will the missing values impact your analyses?

In [226]:
# We can put the values for the ytd_current with month_current and ytd_prior_year with month_prior_year for 2015 to 2017

# Identify the years where ytd_current and ytd_prior_year are entirely missing
missing_years = df_materials_clean.loc[df_materials_clean['ytd_current'].isna(), 'year'].unique()
# missing_years
# Loop through each missing year to calculate and impute YTD values
for year in missing_years:
     # Create a mask for the current year
     year_mask = (df_materials_clean['year'] == year)

     # Calculate cumulative sums for 'ytd_current' and 'ytd_prior_year' for the current year, grouped by 'type'
     # The cumulative sum needs to be calculated within each 'type' group for the given year
     df_materials_clean.loc[year_mask, 'ytd_current'] = df_materials_clean.loc[year_mask].groupby('type')['month_current'].cumsum()
     df_materials_clean.loc[year_mask, 'ytd_prior_year'] = df_materials_clean.loc[year_mask].groupby('type')['month_prior_year'].cumsum()


# # Display a sample of the imputed rows for verification (e.g., year 2015, month 1)
# print("\nVerifying imputation for 2015, month 1 for Grain Products:")
display(df_materials_clean.loc[(df_materials_clean['year'] == 2015) & (df_materials_clean['month'] == 1) & (df_materials_clean['material_type'] == 'Grain Products')])
display(df_materials_clean.loc[(df_materials_clean['year'] == 2015) & (df_materials_clean['month'] == 2) & (df_materials_clean['material_type'] == 'Grain Products')])

# print("\nVerifying imputation for 2015, month 12 for Grain Products:")
display(df_materials_clean.loc[(df_materials_clean['year'] == 2015) & (df_materials_clean['month'] == 12) & (df_materials_clean['material_type'] == 'Grain Products')])

# # Check if there are still null values in ytd_current and ytd_prior_year
print("\nRemaining null values in ytd_current after imputation:")
print(df_materials_clean['ytd_current'].isna().sum())
print("\nRemaining null values in ytd_prior_year after imputation:")
print(df_materials_clean['ytd_prior_year'].isna().sum())


,material_type,year,month,type,month_current,month_prior_year,ytd_current,ytd_prior_year
1008,Grain Products,2015,1,Malt and malt products,314189672,315728713,314189672.0,315728713.0
1009,Grain Products,2015,1,Corn and corn products,59759340,60186876,59759340.0,60186876.0
1010,Grain Products,2015,1,Rice and rice products,52438465,52099471,52438465.0,52099471.0
1011,Grain Products,2015,1,Barley and barley products,8728262,8185386,8728262.0,8185386.0
1012,Grain Products,2015,1,Wheat and wheat products,1316582,1372263,1316582.0,1372263.0


,material_type,year,month,type,month_current,month_prior_year,ytd_current,ytd_prior_year
1020,Grain Products,2015,2,Malt and malt products,292567431,606757103,606757103.0,922485816.0
1021,Grain Products,2015,2,Corn and corn products,49291032,109050372,109050372.0,169237248.0
1022,Grain Products,2015,2,Rice and rice products,48975137,101413602,101413602.0,153513073.0
1023,Grain Products,2015,2,Barley and barley products,9108543,17836805,17836805.0,26022191.0
1024,Grain Products,2015,2,Wheat and wheat products,2060046,3376628,3376628.0,4748891.0


,material_type,year,month,type,month_current,month_prior_year,ytd_current,ytd_prior_year
1140,Grain Products,2015,12,Malt and malt products,155495923,3669211565,3.669212e+09,2.495016e+10
1141,Grain Products,2015,12,Corn and corn products,27270242,570241780,5.702418e+08,4.051625e+09
1142,Grain Products,2015,12,Rice and rice products,36403087,603794296,6.037828e+08,4.084293e+09
1143,Grain Products,2015,12,Barley and barley products,11466970,168839894,1.688399e+08,1.074068e+09
1144,Grain Products,2015,12,Wheat and wheat products,1070326,32145753,3.214575e+07,2.186007e+08



Remaining null values in ytd_current after imputation:
0

Remaining null values in ytd_prior_year after imputation:
0


In [228]:
import pandas as pd

data = {
    'year': [2023, 2023, 2023, 2023, 2023, 2023],
    'month': [1, 2, 1, 2, 1, 2],
    'type': ['Grain A', 'Grain A', 'Grain B', 'Grain B', 'Non-Grain C', 'Non-Grain C'],
    'month_current': [100, 150, 50, 70, 20, 30]
}
df_example = pd.DataFrame(data)

print("Original DataFrame:")
display(df_example)

print("\nDataFrame after groupby('type')['month_current'].cumsum():")
df_example['ytd_calculated'] = df_example.groupby('type')['month_current'].cumsum()
display(df_example)


Original DataFrame:


,year,month,type,month_current
0,2023,1,Grain A,100
1,2023,2,Grain A,150
2,2023,1,Grain B,50
3,2023,2,Grain B,70
4,2023,1,Non-Grain C,20
5,2023,2,Non-Grain C,30



DataFrame after groupby('type')['month_current'].cumsum():


,year,month,type,month_current,ytd_calculated
0,2023,1,Grain A,100,100
1,2023,2,Grain A,150,250
2,2023,1,Grain B,50,50
3,2023,2,Grain B,70,120
4,2023,1,Non-Grain C,20,20
5,2023,2,Non-Grain C,30,50


As you can see, the `ytd_calculated` column correctly calculates the cumulative sum for 'Grain A', 'Grain B', and 'Non-Grain C' independently. This is the same logic applied to `df_materials_clean` for `ytd_current` and `ytd_prior_year`.

In [221]:
df_materials_clean.loc[df_materials_clean['year'] == 2015].groupby('type')['month_current'].cumsum()

,month_current
1008,314189672
1009,59759340
1010,52438465
1011,8728262
1012,1316582
...,...
1144,32145753
1146,847629620
1147,310558359
1148,3611505


In [229]:
df_materials_clean.isna().mean()*100

,0
material_type,0.0
year,0.0
month,0.0
type,0.0
month_current,0.0
month_prior_year,0.0
ytd_current,0.0
ytd_prior_year,0.0


##### **1.3.3** *brewers*
<font color = red>[2 marks]</font> <br>

In [160]:
# Percentage of missing values in columns
df_brewers_clean.isna().mean()*100


,0
year,0.000000
brewer_size,0.000000
n_of_brewers,0.000000
total_barrels,0.793651
taxable_removals,0.000000
total_shipped,1.587302


In [ ]:
# Analyse the missing values and decide what to do with them
# Should you impute with mean or median? Should you drop?

df_brewers_clean

Did you notice that even though there was zero beer production for brewers of size zero barrels, the taxable removals were around 88 thousand.

Why did you think this happened? Analyse this and report the result.
<br>

*Hint*: Taxable removals are the barrels used for consumption within the country/state. These are the barrels that are sold in that year.

### **1.4** Handling Outliers
<font color = red>[5 marks]</font> <br>

In [ ]:
# Check if there are any outliers present



In [ ]:
# Do any columns need standardising?



## **2** Performing Analyses
<font color = red>[100 marks]</font> <br>

### **2.1** *states*
<font color = red>[40 marks]</font>

**Univariate**

There is not much univariate analysis to perform here. You can find out the proportions of instances of different states and the production types (bottles and cans, kegs and barrels etc).

In [ ]:
# Find the number of instances of each state in the data


**Multivariate**

**2.1.0**     <font color = red>[3 marks]</font> <br>
Coming to multivariate analyses, let us first classify the variables into categorical and numerical. Write below the type of variable, whether numerical or categorical and ordered or unordered:

* `state`:
* `year`:
* `barrels`:
* `type`:

Now, start with `states` vs `barrels`.

**2.1.1**     <font color = red>[5 marks]</font> <br>
Calculate the total and average beer production in barrels across all the years for each state. Also find the percentage share of every state in the total production.

In [ ]:
# Find total beer production across all years, and the percentage share of each state in that




In [ ]:
# Find the average beer production per year for all states




**2.1.2**     <font color = red>[3 marks]</font> <br>
Identify the top five and bottom five states and create a histogram showing their average production per year.

In [ ]:
# Identify and visualise top 5 and bottom 5 states



**2.1.3**     <font color = red>[5 marks]</font> <br>
From the total production, visualise the proportion of the production type (kegs and barrels, bottles and cans etc.) Choose a suitable plot to show this information.

In [ ]:
# Visualise the proportions of kegs, bottles and on premises production



**2.1.4**     <font color = red>[3 marks]</font> <br>
Get a description of mean, median, 1/4th and 3/4th quantiles etc for each production type. Compare the means of bottles/cans vs barrels/kegs vs on premises types.

In [ ]:
# Describe mean, quantile values


**2.1.5**     <font color = red>[5 marks]</font> <br>
Create a heatmap of barrels produced by states across years.

In [ ]:
# Create a heatmap of barrels produced by each state across the years



Now, let us analyse the trends in production.

**2.1.6**     <font color = red>[3 marks]</font> <br>
Visualise the variation in beer production across states throughout the years. You don't need to analyse for all the states. Take the top and bottom five.

In [ ]:
# Visualise the trends in production for the most and least productive states



**2.1.7**     <font color = red>[5 marks]</font> <br>
Find the average growth rate per year for the top and bottom states. Do you find a common pattern? Which states are increasing and which states are declining?

In [ ]:
# Calculate the average growth rates of the top and bottom states from the first year to the last year



**2.1.8**     <font color = red>[5 marks]</font> <br>
Analyse the trend of production types across years, irrespective of states. Identify which types are rising or declining or staying constant.

In [ ]:
# Analyse the overall trend of production by production types (cans/barrels etc.)

**2.1.9**     <font color = red>[3 marks]</font> <br>
Now, for the top states, identify which types are rising across years and which are falling.

In [ ]:
# Identify which production types are growing and which are declining



**2.1.10 Results**  
What are your inferences based on the analyses above? What did you find out about trends in production and about different states? Include your findings in the report.


If you want to analyse some more, you can do so below.

### **2.2** *materials*
<font color = red>[30 marks]</font>

In [ ]:
# materials.columns.tolist()

**2.2.0**     <font color = red>[3 marks]</font> <br>
Write below the type of variable, whether numerical or categorical and ordered or unordered, like you did for *states*:

* `1`:
* `2`:
* `3`:
* `4`:
* `5`:

Yearwise production

**2.2.1**     <font color = red>[3 marks]</font> <br>
Calculate the yearwise production of beer from the various types of materials (grain-based and non-grain-based).

In [ ]:
# Calculate the yearly production from grain and non grain materials



**2.2.2**     <font color = red>[4 marks]</font> <br>
Calculate the yearwise production of beer from the various materials (Barley, Malt, Hops etc).

In [ ]:
# Calculate the yearly production from various ingredients



**2.2.3**     <font color = red>[5 marks]</font> <br>
Visualise the proportion of average production from various materials.
Create one visualisation for grain vs non-grain and one for the individual materials like barley, malt, hops etc.

In [ ]:
# Visualise the distribution of average production from various materials and ingredients



**2.2.4**     <font color = red>[5 marks]</font> <br>
What has changed in 10 years? <br>
Create a visualisation to show trends in the use of materials in 2008 vs 2017. Show how the use of grains and non-grain based materials vary over months in 2008 and 2017.

In [ ]:
# Visualise the monthly trends of usage of grain and non grain materials in 2008 vs 2017



**2.2.5**     <font color = red>[5 marks]</font> <br>
Now, for the above visualisation, instead of simply using production quantity, use the difference in current month's production and the production in current month in the prior year.

How did 2008 perform compared to 2007? What about 2017 compared to 2016?

This can be used to identify the months when production is increased seasonally. <br>
For example, if we see that the production in a month is especially high, we can check the difference in production for that month in current year and last year.<br>
If the difference comes out to be small, we can say that the production rises every year in that month. If the difference is large, probably that month was unusualy productive!

**2.2.6**     <font color = red>[5 marks]</font> <br>

So, what do you infer based on the two visualisations above?

**2.2.7 Results**    
Include your insights in the report. What are your inferences on use of different materials across months, production trends and overall material preferences?

### **2.3** brewers
<font color = red>[30 marks]</font> <br>

**2.3.0**     <font color = red>[3 marks]</font> <br>
Write below the type of variable, whether numerical or categorical and ordered or unordered, like you did previously:

* `1`:
* `2`:
* `3`:
* `4`:
* `5`:

**2.3.1**     <font color = red>[5 marks]</font> <br>
Analyze the distribution of brewers by size over time. Are small or large brewers more prevalent in specific years?


In [ ]:
# Analyse the distribution of brewer sizes over time



**2.3.2**     <font color = red>[5 marks]</font> <br>
How does their contribution to total beer production change over time?

In [ ]:
# Analyse the contribution of different brewer sizes to the total production over the years



**2.3.3**     <font color = red>[7 marks]</font> <br>
Investigate the correlation between brewer size and taxable removals. Do larger brewers produce more taxable beer compared to smaller brewers? Also find the correlation value.

In [ ]:
# Find correlation between brewer size and taxable removals
# Note that brewer size is categorical



**2.3.4**     <font color = red>[5 marks]</font> <br>
Visualise `total_shipped` over time to see how the volume of shipped beer fluctuates across different brewer sizes.

In [ ]:
# Visualise the trends in quantities of shipped beer across different brewer sizes



**2.3.5**     <font color = red>[5 marks]</font> <br>
Can you connect consumption with shipped and taxed beer? What did you find out about the distribution of brewers based on their size in the USA?

**2.3.6 Results**

What are your inferences based on the above analyses? Include all your findings in the report.

## **3** Conclusion
<font color = red>[10 marks]</font> <br>

### **3.1** Final Insights and Recommendations
<font color = red>[10 marks]</font> <br>

Conclude your analyses here. Include all the outcomes you found and propose recommendations about different states, brewery sizes and materials suited for production.

Based on your insights, frame a concluding story explaining suitable parameters such as location, size, etc. to be kept in mind while setting up a brewery.

**3.1.0**     <font color = red>[2 marks]</font> <br>
Which material type (malt, corn, rice, barley etc) has the has the highest percentage of current number of barrels for this year/month? Which material type has the least percentage of current number of barrels for this year/month?

**3.1.1**     <font color = red>[2 marks]</font> <br>
In which year and for which brewer size, the total taxable removals were greater than the total barrels of beer produced

**3.1.2**     <font color = red>[2 marks]</font> <br>
What is the average production per year of the top 5 states and the bottom 5 states?  

**3.1.3**     <font color = red>[2 marks]</font> <br>
For which year, the On Premises production was highest and for which state? For which year it is the lowest and for which state?

**3.1.4**     <font color = red>[2 marks]</font> <br>
Which state has a positive year on year growth rate for most of the years starting from 2008 to 2018? Which state has a negative growth rate year on year most of the years?